In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plot_facet_slice(
    ax,
    fs,
    lam,
    dist=10.0,
    ph=0.0,
    th_min=-np.pi/2,
    th_max=np.pi/2,
    n=2000,
    normalize=True,
    dB = False,
    HPBW = False,
    label=None
):
    """
    Plot a polar theta-slice of facet reradiation on a given axis.
    """

    th = np.linspace(th_min, th_max, n)

    def sinc(x):
        return np.where(x == 0, 1.0, np.sin(x) / x)

    k = 2 * np.pi / lam

    arg1 = (np.pi * fs / lam) * np.sin(th) * np.cos(ph)
    arg2 = (np.pi * fs / lam) * np.sin(th) * np.sin(ph)

    #P = ((k * fs**2) / (lam * dist)) \
    #    * sinc(arg1) \
    #    * sinc(arg2)

    P = (fs**2 / lam) \
        * sinc(arg1) \
        * sinc(arg2)
    
    P *= P

    if normalize:
        P /= P.max()

    if dB:
        P = 10 * np.log10(P  / P.max())

    if HPBW and dB:
        HPBW = np.sum(P > -3) * np.degrees(th[1] - th[0])
        print(f"l={lam:.1f} m, fs={fs} m, HPBW = {HPBW:.1f} deg")
        tar_len = 315e3 * np.sin(np.radians(HPBW))
        print(f"Facet visible for {int(tar_len):5d} m in orbit")
    
    ax.plot(th, P, label=label, linewidth=1)

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 6))

lam = 299792458 / 20e6 # wavelength [m]
fs = (500, 250, 100, 50, 25)

for f in fs:

    plot_facet_slice(ax, fs=f, lam=lam, label=f"fs={f} m", dB=True, HPBW=True, dist=315e3)

ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)
ax.set_rmax(1.0)
ax.set_rmin(-40)
ax.set_title("Facet size comparison (normalized)")
ax.legend(loc='best', bbox_to_anchor=(0.7, 0., 0.7, 0.5))
ax.text(np.radians(-105), -10, s="Directivity [dB]")

ax.set_thetamin(-90)
ax.set_thetamax(90)

plt.show()